In [3]:
#!pip install requests BeautifulSoup4

In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

In [2]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [7]:
Hex_Code = []
RGB_Code = []

raw_html = simple_get('https://www.color-hex.com/color-palette/78956')
html = BeautifulSoup(raw_html, 'html.parser')

for i, li in enumerate(html.select('td')):
    if len(li.text) == 7:
        Hex_Code.append(li.text)
    elif len(li.text) >= 11:
        RGB_Code.append(li.text)        

In [8]:
name = html.find('title').text

In [9]:
df_Color = pd.DataFrame(columns=('name','dataColors','background','foreground','tableAccent'))

df_Color = df_Color.append([{'name':name,'dataColors':Hex_Code,'background':Hex_Code[0],'foreground':Hex_Code[4],'tableAccent':Hex_Code[1]}], ignore_index=True)

df_Color.to_json('{}.json'.format(name),
                 orient="records",
                 lines=True)

In [10]:
df_Color

,name,dataColors,background,foreground,tableAccent
0,Crunchy Country Mama Color Palette,"[#ffab8f, #159598, #ffff99, #d74a86, #ffc799]",#ffab8f,#ffc799,#159598
